# RusTorch CoreML Integration - Python Bindings

このノートブックは、PythonバインディングでRusTorchのCoreML機能を使用する方法を示します。

## セットアップとインポート

In [ ]:
# RusTorchのPythonバインディングをインポート
try:
    import rustorch
    print(f"✅ RusTorch version: {rustorch.__version__}")
    print(f"📝 Description: {rustorch.__description__}")
    print(f"👥 Author: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ RusTorchのインポートに失敗: {e}")
    print("maturin develop でビルドしてください")
    exit(1)

import numpy as np
import platform

print(f"🖥️ プラットフォーム: {platform.system()} {platform.release()}")
print(f"🐍 Python version: {platform.python_version()}")

## CoreMLの可用性をチェック

In [ ]:
# CoreML機能の確認
try:
    # CoreMLが利用可能かチェック
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML available: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreMLが利用可能です！")
        
        # デバイス情報を取得
        device_info = rustorch.get_coreml_device_info()
        print("📱 CoreMLデバイス情報:")
        print(device_info)
    else:
        print("⚠️ CoreMLは利用できません")
        if platform.system() != "Darwin":
            print("CoreMLはmacOSでのみ利用可能です")
        else:
            print("CoreMLフィーチャーが有効になっていない可能性があります")
            
except AttributeError:
    print("❌ CoreML関数が見つかりません")
    print("CoreMLフィーチャーでビルドされていない可能性があります")
    coreml_available = False
except Exception as e:
    print(f"❌ CoreMLチェック中にエラー: {e}")
    coreml_available = False

## CoreMLデバイスの作成と操作

In [ ]:
if coreml_available:
    try:
        # CoreMLデバイスを作成
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ CoreMLデバイス作成: {device}")
        
        # デバイス情報を取得
        print(f"🆔 Device ID: {device.device_id()}")
        print(f"✅ Available: {device.is_available()}")
        print(f"💾 Memory limit: {device.memory_limit()} bytes")
        print(f"🧮 Compute units limit: {device.compute_units_limit()}")
        print(f"📚 Model cache size: {device.model_cache_size()}")
        
        # キャッシュのクリーンアップ
        device.cleanup_cache()
        print("🧹 Cache cleaned up")
        
    except Exception as e:
        print(f"❌ CoreMLデバイス操作エラー: {e}")
else:
    print("⚠️ CoreMLが利用できないため、デバイス操作をスキップ")

## CoreMLバックエンドの設定

In [ ]:
if coreml_available:
    try:
        # CoreMLバックエンド設定を作成
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Backend config: {config}")
        
        # 設定値を確認・変更
        print(f"📊 Enable caching: {config.enable_caching}")
        print(f"🗂️ Max cache size: {config.max_cache_size}")
        print(f"📈 Enable profiling: {config.enable_profiling}")
        print(f"🔄 Auto fallback: {config.auto_fallback}")
        
        # 設定を変更
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Updated config: {config}")
        
        # CoreMLバックエンドを作成
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 CoreML backend: {backend}")
        print(f"✅ Backend available: {backend.is_available()}")
        
        # バックエンド統計を取得
        stats = backend.get_stats()
        print(f"📊 Backend stats: {stats}")
        print(f"   Total operations: {stats.total_operations}")
        print(f"   Cache hits: {stats.cache_hits}")
        print(f"   Cache misses: {stats.cache_misses}")
        print(f"   Fallback operations: {stats.fallback_operations}")
        print(f"   Cache hit rate: {stats.cache_hit_rate():.2%}")
        print(f"   Fallback rate: {stats.fallback_rate():.2%}")
        print(f"   Avg execution time: {stats.average_execution_time_ms:.2f}ms")
        
        # キャッシュクリーンアップ
        backend.cleanup_cache()
        print("\n🧹 Backend cache cleaned")
        
    except Exception as e:
        print(f"❌ CoreMLバックエンド操作エラー: {e}")
else:
    print("⚠️ CoreMLが利用できないため、バックエンド操作をスキップ")

## 基本的なテンソル操作（CPU）

CoreMLとの比較のために、まずCPUでの基本操作を実行します。

In [ ]:
try:
    # 基本的なテンソル作成と操作
    print("🧮 基本的なテンソル操作（CPU）")
    
    # NumPy配列からテンソルを作成（簡略化されたインターフェース）
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Matrix A shape: {data_a.shape}")
    print(f"📐 Matrix B shape: {data_b.shape}")
    
    # NumPyで行列乗算（比較用）
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ NumPy matmul result shape: {numpy_result.shape}")
    print(f"📊 Result (first few elements): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 CPU演算完了")
    
except Exception as e:
    print(f"❌ テンソル操作エラー: {e}")

## パフォーマンス比較のシミュレーション

In [ ]:
import time

def benchmark_matrix_operations():
    """異なるサイズの行列でパフォーマンスを比較"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 パフォーマンス比較:")
    print("Size\t\tCPU Time (ms)\tExpected CoreML (ms)")
    print("-" * 50)
    
    for size in sizes:
        # CPU実行時間を測定
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # CoreMLの予想時間（仮定的）
        # 実際の実装では、CoreMLバックエンドからの実測値を使用
        expected_coreml_time = cpu_time * 0.6  # 仮定: CoreMLは40%高速
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 注意: CoreML時間は仮定値です。実際の値は具体的な実装と依存します。")

## デバイス選択のシミュレーション

In [ ]:
def simulate_device_selection():
    """スマートデバイス選択をシミュレート"""
    
    operations = [
        ("小さい行列乗算", (16, 16), "CPU"),
        ("中程度行列乗算", (128, 128), "Metal GPU"),
        ("大きい行列乗算", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("活性化関数", (32, 64, 128, 128), "Metal GPU"),
        ("畳み込み（小）", (1, 3, 32, 32), "CPU"),
        ("畳み込み（大）", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("複素数演算", (128, 128), "Metal GPU"),  # CoreML非対応
        ("統計的分布", (1000,), "CPU"),  # CoreML非対応
    ]
    
    print("🎯 スマートデバイス選択シミュレーション:")
    print("Operation\t\t\tTensor Shape\t\tSelected Device")
    print("-" * 70)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<23}\t{shape_str:<15}\t{device}")
    
    print("\n📝 選択ロジック:")
    print("  • 小さい操作: CPU（オーバーヘッド回避）")
    print("  • 中程度操作: Metal GPU（バランス）")
    print("  • 大きい操作: CoreML（最適化済み）")
    print("  • 非対応操作: GPU/CPUフォールバック")

simulate_device_selection()

## 実践的な使用例: 簡単なニューラルネットワーク層

In [ ]:
def simulate_neural_network_layer():
    """ニューラルネットワーク層のシミュレーション"""
    
    print("🧠 ニューラルネットワーク層シミュレーション:")
    
    # バッチサイズとレイヤー設定
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10クラス
    
    print(f"📊 Batch size: {batch_size}")
    print(f"🔢 Input features: {input_features}")
    print(f"🧮 Hidden features: {hidden_features}")
    print(f"🎯 Output features: {output_features}")
    
    # 前向き伝播のシミュレーション
    steps = [
        ("Input → Hidden", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("ReLU Activation", f"({batch_size}, {hidden_features})", "Metal"),
        ("Hidden → Output", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Forward pass simulation:")
    total_time = 0
    
    for step, shape, device in steps:
        # 仮想的な実行時間（ms）
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ Total forward pass time: {total_time:.2f}ms")
    print(f"🚀 Estimated throughput: {1000/total_time:.0f} batches/second")

simulate_neural_network_layer()

## まとめと次のステップ

In [ ]:
print("📋 RusTorch CoreML統合サマリー:")
print()
print("✅ 完了項目:")
print("  • Jupyter環境の設定")
print("  • RustカーネルとPythonバインディングの作成")
print("  • CoreML可用性チェック")
print("  • デバイス管理と設定")
print("  • バックエンド統計とプロファイリング")
print("  • スマートデバイス選択")
print()
print("🚧 今後の開発:")
print("  • 実際のCoreML演算の実装")
print("  • パフォーマンスベンチマーク")
print("  • より多くの活性化関数とレイヤータイプ")
print("  • エラーハンドリングの改善")
print("  • メモリ最適化")
print()
print("🎯 推奨される次のステップ:")
print("  1. 実際のCoreMLモデルのロードとテスト")
print("  2. MetalとCoreMLのパフォーマンス比較")
print("  3. 実際のディープラーニングワークフローでのテスト")
print("  4. プロダクション環境での評価")

if coreml_available:
    print("\n🎉 おめでとうございます！CoreMLが利用可能で、すべての機能をテストできます。")
else:
    print("\n⚠️ CoreMLが利用できませんが、基本的な機能は動作しています。")
    print("   macOSでCoreMLフィーチャーを有効にしてビルドすることをお勧めします。")